In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision
import torchvision.transforms
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
from torch.utils.data import DataLoader
from torchsummary import summary
from IPython.core.debugger import set_trace

from Tools import Trainer

## Loading datasets

In [2]:
from Tools.Dataloader import datasets_loader
train_loader, valid_loader, test_loader = datasets_loader()

## Hyper-parameters

In [3]:
hyperparams = {'num_epochs': 30,
               'lr0' :0.1, 
               'dropout':0.2,
               'save_checkpoint': False
                  }
cuda_available = torch.cuda.is_available()

## Creating the model

In [4]:
from Classifier.XavierCNN import XavierDropout

model = XavierDropout(hyperparams)
if cuda_available:
    model = model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=hyperparams['lr0'])
criterion = nn.CrossEntropyLoss()

In [ ]:
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]             448
              ReLU-2           [-1, 16, 64, 64]               0
   GaussianDropout-3           [-1, 16, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]           4,640
              ReLU-5           [-1, 32, 64, 64]               0
         MaxPool2d-6           [-1, 32, 32, 32]               0
   GaussianDropout-7           [-1, 32, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          18,496
              ReLU-9           [-1, 64, 32, 32]               0
  GaussianDropout-10           [-1, 64, 32, 32]               0
           Conv2d-11          [-1, 128, 32, 32]          73,856
             ReLU-12          [-1, 128, 32, 32]               0
  GaussianDropout-13          [-1, 128, 32, 32]               0
        MaxPool2d-14          [-1, 128,

## Training 

In [ ]:
from Tools.Trainer import Trainer

trainer = Trainer(model, optimizer, criterion, train_loader, valid_loader, test_loader, hyperparams)
nll_train, nll_valid, acc_train, acc_valid =  trainer.train_model()
print('Done...')

------ Epoch # 1 ------
 Iteration 40: TRAIN 0.6936
 Iteration 80: TRAIN 0.6908
 Iteration 120: TRAIN 0.6883
------ Epoch # 2 ------
 Iteration 160: TRAIN 0.6816
 Iteration 200: TRAIN 0.6796


## Visualization

### accuracy and loss plots

In [ ]:
import Tools.Visualization as V
V.plots(nll_train, nll_valid, acc_train, acc_valid)

### Confusion Matrix

In [ ]:
import Tools.Visualization as V
cnf_matrix = trainer.confusion_matrix()
V.plot_confusion_matrix(cnf_matrix)

## Saving the model

In [ ]:
torch.save(model, "./Experiments/Xavier-Dropout-25-0.1-0.2")



## Generate Submission

In [ ]:
import Tools.Trainer as T
results = T.predict_test_set(model, test_loader)
T.generate_submission(results)